In [6]:
import numpy

In [5]:
import pandas as pd

In [1]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

In [2]:
import sklearn

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [7]:
flights_data = pd.read_csv('Business_Dataset.csv')

/var/folders/qv/2tgf4xk15yl2x3t63qygm7y40000gn/T/ipykernel_77638/353090443.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_data = pd.read_csv('Business_Dataset.csv')


In [ ]:
#date number using quarter and year
flights_data['quarter_year'] = flights_data['quarter']*.25 - .25 + flights_data['Year']

In [11]:
# Define flights with relevant variables

flights = flights_data[['airport_1','airport_2','Geocoded_City1','Geocoded_City2','fare','fare_low','fare_lg', 'quarter_year']]
quarter_year_pairs = flights[['quarter_year']].drop_duplicates().values

In [12]:
# Prepare features for Random Forest
X = flights[['airport_1', 'airport_2', 'quarter_year']]  # You might want to encode these features
y = flights['fare']

# One-hot encoding for categorical variables
X_encoded = pd.get_dummies(X, columns=['airport_1', 'airport_2', 'quarter_year'], drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict fares for all routes in the dataset
flights['predicted_fare'] = rf_model.predict(X_encoded)

# Define flights_dict using predicted fares
flights_dict = {(row['airport_1'], row['airport_2'], row['quarter_year']): row['predicted_fare'] for index, row in flights.iterrows()}

# Create a linear programming problem
problem = LpProblem("Cheapest_Flight_Problem", LpMinimize)

# Define decision variables based on flights_dict
x = LpVariable.dicts("Flight", flights_dict.keys(), lowBound=0, cat='Binary')

# Objective function: Minimize total fare based on predicted_fare
problem += lpSum(flights_dict[key] * x[key] for key in flights_dict.keys()), "Total_Cost"

# Define your source and destination
source = 'ORD'  # Replace with an actual city code
destination = 'JFK'  # Replace with an actual city code

# Add constraints: at least one flight from source and one flight to destination
problem += lpSum(x[(source, airport_2)] for airport_2 in flights[flights['airport_1'] == source]['airport_2']) >= 1, "At_Least_One_Flight_From_Source"
problem += lpSum(x[(city1, destination)] for city1 in flights[flights['airport_2'] == destination]['airport_1']) >= 1, "At_Least_One_Flight_To_Destination"

#Add constraint to make sure same year and quarter
flights_vars = LpVariable.dicts("flight", range(len(flights)), cat='Binary')

for quarter_year in quarter_year_pairs:
    # Select flights that match the current year and quarter
    matching_flights = flights[(flights['quarter_year'] == quarter_year)]
    
    if not matching_flights.empty:
        # Create a constraint for this specific year and quarter
        problem += lpSum(flights_vars[i] for i in matching_flights.index) <= 1  # at most one flight


# Solve the problem
problem.solve()

# Output the results
print("Status:", LpStatus[problem.status])
for key in flights_dict.keys():
    if x[key].varValue > 0:
        print(f"Flight from {key[0]} to {key[1]} with predicted fare: {flights_dict[key]}")

In [ ]:
import folium
from folium import Marker